In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os 
from pprint import pprint

In [3]:
import numpy as np
import decimal
import copy

In [4]:
import streamm

In this getting started example we will calculate the coupling between P3HT oligomers

Now let's create project and resource to keep track of our work

In [5]:
p3ht_et = streamm.Project('P3HT_ET')

And a resource object to keep track of where our files are 

In [6]:
res_local = streamm.Resource('local')

Update relative location of templates directory 

In [7]:
res_local.dir['templates'] =  os.path.join(res_local.dir['home'],'..','templates','')

In [8]:
res_local.make_dir()

In [9]:
p3ht_et.set_resource(res_local)

Now we need to set a remote resource we have ssh access to and run calculations 

In [10]:
peregrine = streamm.Resource('peregrine')

In [11]:
peregrine.meta['type'] = "ssh"
peregrine.ssh['username'] = "tkemper"    
peregrine.ssh['address'] = "peregrine.hpc.nrel.gov"    
peregrine.dir['storage'] = '/mss/users/%s'%(peregrine.ssh['username'])
peregrine.dir['scratch'] = '/scratch/%s'%(peregrine.ssh['username'])
peregrine.dir['home'] = res_local.dir['home']
peregrine.dir['launch'] = res_local.dir['launch']
peregrine.dir['templates'] = res_local.dir['templates']

# Set default simulation specs
peregrine.properties['allocation'] = 'orgopv'
peregrine.properties['e-mail'] = 'travis.kemper@nrel.gov'
peregrine.properties['scratch'] = peregrine.dir['scratch']
peregrine.properties['allocation'] = 'orgopv'
peregrine.properties['walltime'] = 4
peregrine.properties['nodes'] = int(1)
peregrine.properties['ppn'] = int(24)
peregrine.properties['nproc'] = peregrine.properties['nodes']*peregrine.properties['ppn']
peregrine.properties['queue'] = 'short'
peregrine.properties['feature'] = '24core'
peregrine.properties['exe_command'] = 'qsub '

In [12]:
pprint(peregrine.properties)

{u'allocation': 'orgopv',
 'e-mail': 'travis.kemper@nrel.gov',
 u'exe_command': 'qsub ',
 u'feature': '24core',
 u'nodes': 1,
 u'nproc': 24,
 u'pmem': 1500,
 u'ppn': 24,
 u'queue': 'short',
 'scratch': '/scratch/tkemper',
 u'walltime': 4}


In [13]:
peregrine.make_dir()

Explicitely create a thiophene molecule

In [14]:
bbTh = streamm.Buildingblock('thiophene')
symbols = ['C','C','C','C','S','H','H','H','H']
positions = [ ]
positions.append([-1.55498576,-1.91131218,-0.00081000])
positions.append([-0.17775976,-1.91131218,-0.00081000])
positions.append([0.34761524,-0.57904218,-0.00081000])
positions.append([-0.65884476,0.36101082,0.00000000])
positions.append([-2.16948076,-0.35614618,-0.00000800])
positions.append([-2.18966076,-2.79526518,-0.00132100])
positions.append([0.45389024,-2.80145418,-0.00106400])
positions.append([1.41682424,-0.35961818,-0.00138200])
positions.append([-0.51943676,1.44024682,0.00064700])
for i in range(len(symbols)):
    pt_i = streamm.Particle(symbol=symbols[i])
    pos_i = positions[i]
    bbTh.add_partpos(pt_i,pos_i)

In [15]:
bbTh.particles[5].rsite = 'termcap'
bbTh.particles[6].rsite = 'funccap'
bbTh.particles[8].rsite = 'termcap'

Set some properties of the molecule to keep track of the parts

In [16]:
c_cnt =1
h_cnt =1

for pkey_i, particle_i  in bbTh.particles.iteritems():
            if( particle_i.symbol == 'C' ):
                particle_i.label = "C%d"%(c_cnt)

                particle_i.resname = "SCP2"
                particle_i.residue = 1
                particle_i.charge = -0.025

                c_cnt +=1 
            if( particle_i.symbol == 'S' ):
                particle_i.resname = "ThS"
                particle_i.residue = 2
                particle_i.charge = -0.3

            if( particle_i.symbol == 'H' ):
                particle_i.label = "H%d"%(h_cnt)

                particle_i.resname = "HA"
                particle_i.residue = 3
                particle_i.charge = 0.1

                h_cnt +=1 
                

Set the force-field type and guess some reasonable charges 

In [17]:
for pkey_i, particle_i  in bbTh.particles.iteritems():
            if( particle_i.symbol == 'C' ):
                particle_i.paramkey = 'CA'
                particle_i.charge = -0.025
            if( particle_i.symbol == 'S' ):
                particle_i.paramkey = 'S'
                particle_i.charge = -0.3
            if( particle_i.symbol == 'H' ):
                particle_i.paramkey = 'HA'
                particle_i.charge = 0.1

Optimize structure with NWChem

In [18]:
nwchem_i = streamm.NWChem('nw_opt_{}'.format(bbTh.tag))

Add thiophene structure 

In [19]:
nwchem_i.strucC = copy.deepcopy(bbTh)

Set calculation to run on external resource

In [20]:
nwchem_i.set_resource(peregrine)

Add calculation to project

In [21]:
p3ht_et.add_calc(nwchem_i)

Make the local directories 

In [22]:
nwchem_i.make_dir()

Change to the `launch` directory

In [23]:
print nwchem_i.dir['launch']

/Users/tkemper/Development/streamm-tools/examples/scratch/nw_opt_thiophene/


In [24]:
os.chdir(nwchem_i.dir['launch'])

In [25]:
file_type = 'templates'
file_key = 'run'
file_name = "nwchem_peregrine.pbs"
from_dirkey = 'templates'
to_dirkey = 'launch'
nwchem_i.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)

In [26]:
file_type = 'templates'
file_key = 'nw'
file_name = "nwchem.nw"
from_dirkey = 'templates'
to_dirkey = 'launch'
nwchem_i.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)

In [27]:
nwchem_i.load_str('templates','nw')        
nwchem_i.load_str('templates','run')

In [28]:
nwchem_i.properties['basis'] = '6-31g'
nwchem_i.properties['method'] = 'UHF'
nwchem_i.properties['charge'] = 0
nwchem_i.properties['spin_mult'] = 1
nwchem_i.properties['task'] = 'SCF optimize'
nwchem_i.properties['coord'] = nwchem_i.strucC.write_coord()

In [29]:
pprint(nwchem_i.properties)

{u'allocation': 'orgopv',
 'basis': '6-31g',
 'charge': 0,
 'comp_key': 'compressed',
 'compress': 'tar -czf ',
 'compress_sufix': 'tgz',
 'coord': u'     C      -1.55498576      -1.91131218      -0.00081000 \n     C      -0.17775976      -1.91131218      -0.00081000 \n     C       0.34761524      -0.57904218      -0.00081000 \n     C      -0.65884476       0.36101082       0.00000000 \n     S      -2.16948076      -0.35614618      -0.00000800 \n     H      -2.18966076      -2.79526518      -0.00132100 \n     H       0.45389024      -2.80145418      -0.00106400 \n     H       1.41682424      -0.35961818      -0.00138200 \n     H      -0.51943676       1.44024682       0.00064700 \n',
 'e-mail': 'travis.kemper@nrel.gov',
 u'exe_command': 'qsub ',
 u'feature': '24core',
 u'finish_str': u'Total times  cpu:',
 'method': 'UHF',
 u'nodes': 1,
 u'nproc': 24,
 u'pmem': 1500,
 u'ppn': 24,
 u'queue': 'short',
 'scratch': u'/scratch/tkemper/nw_opt_thiophene/',
 'spin_mult': 1,
 'task': 'SCF optim

In [30]:
nwchem_i.replacewrite_prop('nw','input','nw','%s.nw'%(nwchem_i.tag))

In [31]:
nwchem_i.properties['input_nw'] = nwchem_i.files['input']['nw']
nwchem_i.replacewrite_prop('run','scripts','run','%s.pbs'%(nwchem_i.tag))

In [32]:
file_type = 'output'
file_key = 'log'
file_name = "%s.log"%(nwchem_i.tag)
nwchem_i.add_file(file_type,file_key,file_name)

Save details in .json files 

In [33]:
os.chdir(nwchem_i.dir['home'])
p3ht_et.dump_json()

In [34]:
os.chdir(nwchem_i.dir['launch'])

In [35]:
nwchem_i.push()

In [36]:
nwchem_i.run()

Check status unit finished

In [37]:
nwchem_i.check()
print nwchem_i.meta['status']

finished


In [38]:
nwchem_i.analysis()

Print energies 

In [39]:
print nwchem_i.properties['alpha_energies'][10:20]
print nwchem_i.properties['energy']

[-0.4732391, -0.3475018, -0.3290008, 0.1318747, 0.2130991, 0.2315597, 0.2416684, 0.269473, 0.2970013, 0.3198832]
-551.190758104


Check that the positions of the structure have been optimized 

In [40]:
for pk,p in bbTh.particles.iteritems():
    print pk,p.symbol,bbTh.positions[pk]

0 C [ -1.55498576e+00  -1.91131218e+00  -8.10000000e-04]
1 C [ -1.77759760e-01  -1.91131218e+00  -8.10000000e-04]
2 C [ 0.34761524 -0.57904218 -0.00081   ]
3 C [-0.65884476  0.36101082  0.        ]
4 S [ -2.16948076e+00  -3.56146180e-01  -8.00000000e-06]
5 H [ -2.18966076e+00  -2.79526518e+00  -1.32100000e-03]
6 H [  4.53890240e-01  -2.80145418e+00  -1.06400000e-03]
7 H [  1.41682424e+00  -3.59618180e-01  -1.38200000e-03]
8 H [ -5.19436760e-01   1.44024682e+00   6.47000000e-04]


In [41]:
print nwchem_i.strucC.positions

[[ -1.52896184e+00  -1.96220294e+00  -8.01740000e-04]
 [ -1.87314220e-01  -1.91410534e+00  -1.04619000e-03]
 [  3.42508490e-01  -5.70353440e-01  -6.76450000e-04]
 [ -6.05169850e-01   3.80532190e-01  -4.87700000e-05]
 [ -2.25058851e+00  -3.24210270e-01   2.87000000e-05]
 [ -2.16222169e+00  -2.82020399e+00  -1.04323000e-03]
 [  4.31356700e-01  -2.78797334e+00  -1.43211000e-03]
 [  1.39104061e+00  -3.54125130e-01  -9.23550000e-04]
 [ -4.82488530e-01   1.43974965e+00   3.85340000e-04]]


Store the results in a tar ball in the storage directory 

In [42]:
nwchem_i.store()

Now let us calculate the ESP charges to use in our forcefield 

In [43]:
nwchem_esp = streamm.NWChem('nw_esp_{}'.format(bbTh.tag))

Add thiophene structure with optimized coordinates from previous calculation

In [44]:
nwchem_esp.strucC = copy.deepcopy(nwchem_i.strucC)

Set calculation to run on external resource

In [45]:
nwchem_esp.set_resource(peregrine)

Add calculation to project

In [46]:
p3ht_et.add_calc(nwchem_esp)

Make the local directories 

In [47]:
nwchem_esp.make_dir()

Change to the `launch` directory

In [48]:
print nwchem_esp.dir['launch']

/Users/tkemper/Development/streamm-tools/examples/scratch/nw_esp_thiophene/


In [49]:
os.chdir(nwchem_esp.dir['launch'])

In [50]:
file_type = 'templates'
file_key = 'run'
file_name = "nwchem_peregrine.pbs"
from_dirkey = 'templates'
to_dirkey = 'launch'
nwchem_esp.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)

In [51]:
file_type = 'templates'
file_key = 'nw'
file_name = "nwchem.nw"
from_dirkey = 'templates'
to_dirkey = 'launch'
nwchem_esp.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)

In [52]:
nwchem_esp.load_str('templates','nw')        
nwchem_esp.load_str('templates','run')

In [53]:
nwchem_esp.properties['basis'] = '6-31g'
nwchem_esp.properties['method'] = 'UHF'
nwchem_esp.properties['charge'] = 0
nwchem_esp.properties['spin_mult'] = 1
nwchem_esp.properties['task'] = 'esp'
nwchem_esp.properties['coord'] = nwchem_esp.strucC.write_coord()

In [54]:
pprint(nwchem_esp.properties)

{u'allocation': 'orgopv',
 'basis': '6-31g',
 'charge': 0,
 'comp_key': 'compressed',
 'compress': 'tar -czf ',
 'compress_sufix': 'tgz',
 'coord': u'     C      -1.52896184      -1.96220294      -0.00080174 \n     C      -0.18731422      -1.91410534      -0.00104619 \n     C       0.34250849      -0.57035344      -0.00067645 \n     C      -0.60516985       0.38053219      -0.00004877 \n     S      -2.25058851      -0.32421027       0.00002870 \n     H      -2.16222169      -2.82020399      -0.00104323 \n     H       0.43135670      -2.78797334      -0.00143211 \n     H       1.39104061      -0.35412513      -0.00092355 \n     H      -0.48248853       1.43974965       0.00038534 \n',
 'e-mail': 'travis.kemper@nrel.gov',
 u'exe_command': 'qsub ',
 u'feature': '24core',
 u'finish_str': u'Total times  cpu:',
 'method': 'UHF',
 u'nodes': 1,
 u'nproc': 24,
 u'pmem': 1500,
 u'ppn': 24,
 u'queue': 'short',
 'scratch': u'/scratch/tkemper/nw_esp_thiophene/',
 'spin_mult': 1,
 'task': 'esp',
 'u

In [55]:
nwchem_esp.replacewrite_prop('nw','input','nw','%s.nw'%(nwchem_esp.tag))

In [56]:
nwchem_esp.properties['input_nw'] = nwchem_esp.files['input']['nw']
nwchem_esp.replacewrite_prop('run','scripts','run','%s.pbs'%(nwchem_esp.tag))

In [57]:
file_type = 'output'
file_key = 'log'
file_name = "%s.log"%(nwchem_esp.tag)
nwchem_esp.add_file(file_type,file_key,file_name)

Save details in .json files 

In [58]:
os.chdir(nwchem_esp.dir['home'])
p3ht_et.dump_json()

In [59]:
os.chdir(nwchem_esp.dir['launch'])

In [60]:
nwchem_esp.push()

In [61]:
nwchem_esp.run()

Check status unit finished

In [62]:
nwchem_esp.check()
print nwchem_esp.meta['status']

written


In [63]:
nwchem_esp.analysis()

Print energies 

In [64]:
print nwchem_esp.properties['alpha_energies'][10:20]
print nwchem_esp.properties['energy']

[]
0.0


Check that the positions of the structure have been optimized 

In [65]:
for pk,p in nwchem_esp.strucC.particles.iteritems():
    print pk,p.symbol,p.charge

0 C -0.025
1 C -0.025
2 C -0.025
3 C -0.025
4 S -0.3
5 H 0.1
6 H 0.1
7 H 0.1
8 H 0.1


Store the results in a tar ball in the storage directory 

In [ ]:
nwchem_esp.store()

In [ ]:
nwchem_esp.pull()

Build hexane

In [66]:
bbHex = streamm.Buildingblock('hexane')
symbols = ['C','H','H','H','C','H','H','C','H','H','C','H','H','C','H','H','C','H','H','H']
positions = [ ]
positions.append([-6.410969,-0.381641,-0.000031])
positions.append([-7.310084,0.245311,-0.000038])
positions.append([-6.456117,-1.028799,0.884636])
positions.append([-6.456111,-1.028812,-0.884689])
positions.append([-5.135268,0.467175,-0.000033])
positions.append([-5.135484,1.128782,0.877977])
positions.append([-5.135479,1.128771,-0.87805])
positions.append([-3.850566,-0.371258,-0.000024])
positions.append([-3.85112,-1.033978,0.87841])
positions.append([-3.851114,-1.033987,-0.878451])
positions.append([-2.567451,0.469603,-0.000024])
positions.append([-2.567784,1.132155,0.8784])
positions.append([-2.567776,1.132146,-0.878455])
positions.append([-1.283527,-0.370234,-0.000013])
positions.append([-1.28337,-1.032804,0.87836])
positions.append([-1.28336,-1.032812,-0.87838])
positions.append([0.00482234,0.47342231,-0.00000898])
positions.append([0.02595107,1.09220686,0.87266464])
positions.append([0.85585781,-0.17514133,0.00194589])
positions.append([0.02780957,1.08937798,-0.87463473])
for i in range(len(symbols)):
    pt_i = streamm.Particle(symbol=symbols[i])
    pos_i = positions[i]
    bbHex.add_partpos(pt_i,pos_i)

In [68]:
bbHex.particles[0].rsite = 'rg'
bbHex.particles[1].rsite = 'rgcap'

In [69]:
c_cnt =1
h_cnt =1
for pkey_i, particle_i  in bbHex.particles.iteritems():
            if( particle_i.symbol == 'C' ):
                particle_i.label = "C%d"%(c_cnt)
                particle_i.resname = "SCP3"
                particle_i.residue = c_cnt
                c_cnt +=1 
            if( particle_i.symbol == 'H' ):
                particle_i.label = "H%d"%(h_cnt)
                particle_i.resname = "HC"
                particle_i.residue = c_cnt -1 
                h_cnt +=1 

Set the parameter keys and some reasonable atomic charges 

In [85]:
for pkey_i, particle_i  in bbHex.particles.iteritems():
            if( particle_i.symbol == 'C' ):
                particle_i.paramkey = 'CT'
                particle_i.charge = -0.12

            if( particle_i.symbol == 'H' ):
                particle_i.paramkey = 'HC'
                particle_i.charge = 0.06
            print pkey_i, particle_i.symbol,particle_i.charge

0 C -0.12
1 H 0.06
2 H 0.06
3 H 0.06
4 C -0.12
5 H 0.06
6 H 0.06
7 C -0.12
8 H 0.06
9 H 0.06
10 C -0.12
11 H 0.06
12 H 0.06
13 C -0.12
14 H 0.06
15 H 0.06
16 C -0.12
17 H 0.06
18 H 0.06
19 H 0.06


In [86]:
bbHex.particles[0].charge  = -0.18
bbHex.particles[16].charge  = -0.18

In [87]:
total_charge = 0.0
for pkey_i, particle_i  in bbHex.particles.iteritems():
            total_charge += particle_i.charge 

Check that the molecule is neutral 

In [91]:
print total_charge

0.0


Looks good let's set up some calculations

In [ ]:
calc_i = streamm.Gaussian('g_methane_HF')

In [ ]:
Methane_example.add_calc(calc_i) # Add it to the project 

In [ ]:
calc_i.strucC = ME               # set the strucC to the structure container 

In [ ]:
print calc_i.tag

Let's use the remote resource to run this calculation 

In [ ]:
calc_i.set_resource(peregrine)

* home : directory is the root directory for the calculation/project
* template : directory of template input and run files to be modified to run the calculation 
* materials : directory to store structure files (.xyz) 
* launch : directory to temporarly store files before they are copied to the remote resouce 
* scratch : directory to run the calculation
* storage : directory to store completed calculation data

In [ ]:
pprint(calc_i.dir)

In [ ]:
calc_i.make_dir()

In [ ]:
print calc_i.dir['launch']

In [ ]:
os.chdir(calc_i.dir['launch'])

In [ ]:
file_type = 'templates'
file_key = 'run'
file_name = "gaussian_peregrine2.pbs"
from_dirkey = 'templates'
to_dirkey = 'launch'
calc_i.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)

In [ ]:
file_type = 'templates'
file_key = 'com'
file_name = "gaussian.com"
from_dirkey = 'templates'
to_dirkey = 'launch'
calc_i.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)

In [ ]:
calc_i.load_str('templates','com')        
calc_i.load_str('templates','run')

In [ ]:
calc_i.properties['commands'] = 'HF/3-21G SP'
calc_i.properties['charge'] = 0
calc_i.properties['spin_mult'] = 1
calc_i.properties['coord'] = calc_i.strucC.write_coord()

In [ ]:
calc_i.replacewrite_prop('com','input','com','%s.com'%(calc_i.tag))

In [ ]:
calc_i.properties['input_com'] = calc_i.files['input']['com']
calc_i.replacewrite_prop('run','scripts','run','%s.pbs'%(calc_i.tag))

In [ ]:
file_type = 'output'
file_key = 'log'
file_name = "%s.log"%(calc_i.tag)
calc_i.add_file(file_type,file_key,file_name)

In [ ]:
file_type = 'output'
file_key = 'fchk'
file_name = "%s.fchk"%(calc_i.tag)
calc_i.add_file(file_type,file_key,file_name)

Save details in .json files 

In [ ]:
os.chdir(calc_i.dir['home'])
Methane_example.dump_json()

In [ ]:
os.chdir(calc_i.dir['launch'])

In [ ]:
calc_i.push()

In [ ]:
calc_i.run()

Cool. While that is in the queue let's setup some more jobs

Let's also run a NWChem calculation

In [ ]:
nwchem_i = streamm.NWChem('nw_methane_HF')

In [ ]:
Methane_example.add_calc(nwchem_i)

In [ ]:
nwchem_i.strucC = ME

In [ ]:
print nwchem_i.tag

In [ ]:
nwchem_i.set_resource(peregrine)

In [ ]:
pprint(nwchem_i.properties['scratch'])

In [ ]:
nwchem_i.make_dir()

In [ ]:
print nwchem_i.dir['launch']

In [ ]:
os.chdir(nwchem_i.dir['launch'])

In [ ]:
file_type = 'templates'
file_key = 'run'
file_name = "nwchem_peregrine.pbs"
from_dirkey = 'templates'
to_dirkey = 'launch'
nwchem_i.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)

In [ ]:
file_type = 'templates'
file_key = 'nw'
file_name = "nwchem.nw"
from_dirkey = 'templates'
to_dirkey = 'launch'
nwchem_i.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)

In [ ]:
nwchem_i.load_str('templates','nw')        
nwchem_i.load_str('templates','run')

In [ ]:
nwchem_i.properties['basis'] = '6-31g'
nwchem_i.properties['method'] = 'UHF'
nwchem_i.properties['charge'] = 0
nwchem_i.properties['spin_mult'] = 1
nwchem_opt.properties['task'] = 'SCF '
nwchem_i.properties['coord'] = nwchem_i.strucC.write_coord()

In [ ]:
pprint(nwchem_i.properties)

In [ ]:
nwchem_i.replacewrite_prop('nw','input','nw','%s.nw'%(nwchem_i.tag))

In [ ]:
nwchem_i.properties['input_nw'] = nwchem_i.files['input']['nw']
nwchem_i.replacewrite_prop('run','scripts','run','%s.pbs'%(nwchem_i.tag))

In [ ]:
file_type = 'output'
file_key = 'log'
file_name = "%s.log"%(nwchem_i.tag)
nwchem_i.add_file(file_type,file_key,file_name)

Save details in .json files 

In [ ]:
os.chdir(nwchem_i.dir['home'])
Methane_example.dump_json()

In [ ]:
os.chdir(nwchem_i.dir['launch'])

In [ ]:
nwchem_i.push()

In [ ]:
nwchem_i.run()

Okay we have a couple 

In [ ]:
Methane_example.check()

In [ ]:
nwchem_i.analysis()

In [ ]:
print nwchem_i.properties['alpha_energies']

In [ ]:
print nwchem_i.properties['N_alpha_occ']

In [ ]:
Methane_example.store()

In [ ]:
Methane_example.pull()

In [ ]:
os.chdir(nwchem_i.dir['home'])
Methane_example.dump_json()

Neat!

Now let's optimize the structure and calculate the ESP charges

In [ ]:
nwchem_opt = streamm.NWChem('nw_methane_OPT')

In [ ]:
Methane_example.add_calc(nwchem_opt)

In [ ]:
nwchem_opt.strucC = ME

In [ ]:
print nwchem_opt.tag

In [ ]:
nwchem_opt.set_resource(peregrine)

In [ ]:
pprint(nwchem_opt.properties['scratch'])

In [ ]:
nwchem_opt.make_dir()

In [ ]:
print nwchem_opt.dir['launch']

In [ ]:
os.chdir(nwchem_opt.dir['launch'])

In [ ]:
file_type = 'templates'
file_key = 'run'
file_name = "nwchem_peregrine.pbs"
from_dirkey = 'templates'
to_dirkey = 'launch'
nwchem_opt.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)

In [ ]:
file_type = 'templates'
file_key = 'nw'
file_name = "nwchem.nw"
from_dirkey = 'templates'
to_dirkey = 'launch'
nwchem_opt.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)

In [ ]:
nwchem_opt.load_str('templates','nw')        
nwchem_opt.load_str('templates','run')

In [ ]:
nwchem_opt.properties['basis'] = '6-31g'
nwchem_opt.properties['method'] = 'UHF'
nwchem_opt.properties['charge'] = 0
nwchem_opt.properties['spin_mult'] = 1
nwchem_opt.properties['task'] = 'SCF optimize'
nwchem_opt.properties['coord'] = nwchem_opt.strucC.write_coord()

In [ ]:
pprint(nwchem_opt.properties)

In [ ]:
nwchem_opt.replacewrite_prop('nw','input','nw','%s.nw'%(nwchem_opt.tag))

In [ ]:
nwchem_opt.properties['input_nw'] = nwchem_opt.files['input']['nw']
nwchem_opt.replacewrite_prop('run','scripts','run','%s.pbs'%(nwchem_opt.tag))

In [ ]:
file_type = 'output'
file_key = 'log'
file_name = "%s.log"%(nwchem_opt.tag)
nwchem_opt.add_file(file_type,file_key,file_name)

Save details in .json files 

In [ ]:
os.chdir(nwchem_opt.dir['home'])
Methane_example.dump_json()

In [ ]:
os.chdir(nwchem_opt.dir['launch'])

In [ ]:
nwchem_opt.push()

In [ ]:
nwchem_opt.run()

In [ ]:
nwchem_opt.check()

In [ ]:
print nwchem_opt.meta

In [ ]:
Methane_example.check()

In [ ]:
Methane_example.store()

In [ ]:
Methane_example.pull()

In [ ]:
nwchem_opt.analysis()

In [ ]:
print nwchem_opt.strucC.write_xyz_str()

In [ ]:
os.chdir(nwchem_opt.dir['materials'])

In [ ]:
nwchem_opt.strucC.tag = '{}_{}'.format(nwchem_opt.strucC.tag,nwchem_opt.tag)

In [ ]:
nwchem_opt.strucC.write_xyz()

In [ ]:
nwchem_esp = streamm.NWChem('nw_methane_ESP')

In [ ]:
ME_OPT = streamm.Buildingblock('methane_nw_methane_OPT')

In [ ]:
ME_OPT.read_xyz()

In [ ]:
print(ME.write_xyz_str())

In [ ]:
Methane_example.add_calc(nwchem_esp)

In [ ]:
nwchem_esp.strucC = ME_OPT

In [ ]:
print nwchem_esp.tag

In [ ]:
nwchem_esp.set_resource(peregrine)

In [ ]:
pprint(nwchem_esp.properties['scratch'])

In [ ]:
nwchem_esp.make_dir()

In [ ]:
print nwchem_esp.dir['launch']

In [ ]:
os.chdir(nwchem_esp.dir['launch'])

In [ ]:
file_type = 'templates'
file_key = 'run'
file_name = "nwchem_peregrine.pbs"
from_dirkey = 'templates'
to_dirkey = 'launch'
nwchem_esp.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)

In [ ]:
file_type = 'templates'
file_key = 'nw'
file_name = "nwchem.nw"
from_dirkey = 'templates'
to_dirkey = 'launch'
nwchem_esp.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)

In [ ]:
nwchem_esp.load_str('templates','nw')        
nwchem_esp.load_str('templates','run')

In [ ]:
nwchem_esp.properties['basis'] = '6-31g'
nwchem_esp.properties['method'] = 'UHF'
nwchem_esp.properties['charge'] = 0
nwchem_esp.properties['spin_mult'] = 1
nwchem_esp.properties['task'] = 'esp'
nwchem_esp.properties['coord'] = nwchem_esp.strucC.write_coord()

In [ ]:
pprint(nwchem_esp.properties)

In [ ]:
nwchem_esp.replacewrite_prop('nw','input','nw','%s.nw'%(nwchem_esp.tag))

In [ ]:
nwchem_esp.properties['input_nw'] = nwchem_esp.files['input']['nw']
nwchem_esp.replacewrite_prop('run','scripts','run','%s.pbs'%(nwchem_esp.tag))

In [ ]:
file_type = 'output'
file_key = 'log'
file_name = "%s.log"%(nwchem_esp.tag)
nwchem_esp.add_file(file_type,file_key,file_name)

Save details in .json files 

In [ ]:
os.chdir(nwchem_esp.dir['home'])
Methane_example.dump_json()

In [ ]:
os.chdir(nwchem_esp.dir['launch'])

In [ ]:
nwchem_esp.push()

In [ ]:
nwchem_esp.run()

In [ ]:
print nwchem_esp.tag,nwchem_esp.files['output']

In [ ]:
nwchem_esp.check()

In [ ]:
print nwchem_esp.meta

In [ ]:
Methane_example.check()

In [ ]:
Methane_example.store()

In [ ]:
Methane_example.pull()

In [ ]:
nwchem_opt.analysis()

Now we have an optimized molecular geometery and ESP charges